<a href="https://colab.research.google.com/github/21goldy/DOS_Detection---Project-Exhibition-1/blob/main/Detection_of_Slowloris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Detection & Prevention of the Slow loris attack utilizing the IsolationForest algorithm and HTTP port blocking - Group 16 { display-mode: "form" }
# Detection & Prevention of the Slow loris attack utilizing the IsolationForest algorithm and HTTP port blocking - Group 16

In [ ]:
# importing the required modules of python 

import io
import pandas as pd
from google.colab import files

In [ ]:
upload_file = files.upload() # uploading the collected CSV from wireshark for DOS analysis and attack detection 

Saving detect_test3.csv to detect_test3.csv


In [ ]:
for fn in upload_file.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(upload_file[fn])))

User uploaded file "detect_test3.csv" with length 33475 bytes


In [ ]:
data = pd.read_csv(io.StringIO(upload_file['detect_test3.csv'].decode('utf-8'))) # reading the CSV file and storing it under the name "data" 

In [ ]:
data.shape # defining the total rows and columns present (Here total rows can also be understood as the number of transmissions captured)

(280, 7)

In [ ]:
transmission_count = data.shape[0] # storing the rows/transmissions captured as "transmission_count"

In [ ]:
data.head() # representing the starting 5 data sets in "data"

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,10.0.2.15,192.168.43.1,DNS,95,Standard query 0x4cea A content-signature-2.cd...
1,2,0.000032,10.0.2.15,192.168.43.1,DNS,95,Standard query 0x88ef AAAA content-signature-2...
2,3,0.006516,192.168.43.1,10.0.2.15,DNS,362,Standard query response 0x88ef AAAA content-si...
3,4,0.073744,192.168.43.1,10.0.2.15,DNS,336,Standard query response 0x4cea A content-signa...
4,5,0.879856,10.0.2.15,13.227.214.62,TCP,74,33446 > 443 [SYN] Seq=0 Win=64240 Len=0 MSS=...


In [ ]:
data.tail() # representing the ending 5 data sets in "data"

,No.,Time,Source,Destination,Protocol,Length,Info
275,276,52.548113,52.84.6.10,10.0.2.15,TCP,60,[TCP Keep-Alive ACK] 443 > 33846 [ACK] Seq=4...
276,277,54.596082,10.0.2.15,117.18.237.29,TCP,54,[TCP Keep-Alive] 41910 > 80 [ACK] Seq=371 Ac...
277,278,54.596144,10.0.2.15,117.18.237.29,TCP,54,[TCP Keep-Alive] 41908 > 80 [ACK] Seq=371 Ac...
278,279,54.596804,117.18.237.29,10.0.2.15,TCP,60,[TCP Keep-Alive ACK] 80 > 41910 [ACK] Seq=80...
279,280,54.596804,117.18.237.29,10.0.2.15,TCP,60,[TCP Keep-Alive ACK] 80 > 41908 [ACK] Seq=80...


In [ ]:
data.describe() # description of the data present in the dataset "data"

,No.,Time,Length
count,280.000000,280.000000,280.000000
mean,140.500000,7.109544,236.671429
std,80.973247,10.884204,405.362273
min,1.000000,0.000000,42.000000
25%,70.750000,1.751323,54.000000
50%,140.500000,3.778027,60.000000
75%,210.250000,5.736076,118.000000
max,280.000000,54.596804,2047.000000


In [ ]:
data.Source # listing the Source IP's present in the captured file

0          10.0.2.15
1          10.0.2.15
2       192.168.43.1
3       192.168.43.1
4          10.0.2.15
           ...      
275       52.84.6.10
276        10.0.2.15
277        10.0.2.15
278    117.18.237.29
279    117.18.237.29
Name: Source, Length: 280, dtype: object

In [ ]:
source = data.Source 
source.nunique() # finding the unique IP's present in the Source column

12

In [ ]:
destination = data.Destination
destination.nunique() # finding the unique IP's present in the Destination column

12

In [ ]:
new_source = source.drop_duplicates() # removing the all the duplicates from the Source column
source_list = list(new_source) # creating list of all the unique IP's 
source_list
print(len(source_list))

12


In [ ]:
major_ips = [] # empty list for keeping the majorly participating IP
minor_ips = [] # empty list for keeping the less participating IP
for i in source_list:
  analyse_ip = data[data.Source == i]
  transmission_of_ip = analyse_ip.shape[0]
  set_criteria = (transmission_count * 35)/100
  # print(set_criteria)
  if transmission_of_ip > set_criteria:
    major_ips.append(i)
  else:
    minor_ips.append(i)

In [ ]:
major_ips # list of actively participating ip

['10.0.2.15']

In [ ]:
minor_ips # list of less active ip

['192.168.43.1',
 '13.227.214.62',
 '52.84.6.10',
 '52.84.6.14',
 '34.215.101.125',
 '117.18.237.29',
 '104.18.10.207',
 '104.16.19.94',
 '69.16.175.42',
 'PcsCompu_00:75:4e',
 'RealtekU_12:35:02']

In [ ]:
#@title Analysis of the Major IPs { display-mode: "form" }


In [ ]:
for i in major_ips:
  dest_list_of_major_ip = list(data[data.Source == i].Destination.drop_duplicates()) # transmitted destination list of major IP
  major_df = pd.DataFrame(
      [
       (
           i,
           dest_list_of_major_ip,
           f'{data[data.Source == i].Protocol.max()}',
           f'{data[data.Source == i].Length.sum()}',
           'Attack'
       )
      ],
      columns=(
          'Source', 'Destination', 'Max_Protocol', 'Total_Length', 'Result'
          )
                 )
  major_df.to_csv (f'result_of_{i}.csv', index = False, header=True) # writing data in the csv file "result_of_{ip}"
  print(major_df.head())
  print(major_df.Destination)

      Source  ...  Result
0  10.0.2.15  ...  Attack

[1 rows x 5 columns]
0    [192.168.43.1, 13.227.214.62, 52.84.6.10, 52.8...
Name: Destination, dtype: object


In [ ]:
a = data[data.Source == '10.0.2.15'] 
b = a[a.Destination == '192.185.184.157']
b # transmission between the major IP "a" = '10.0.2.15' to "b" = '192.185.184.157'

b.to_csv (r'transmission_1.csv', index = False, header=True) # writing data in the csv file "transmission_1"

In [ ]:
c = data[data.Source == '192.185.184.157']
d = c[c.Destination == '10.0.2.15']
d # transmission between the major IP "c" = '192.185.184.157' to "d" = '10.0.2.15'

d.to_csv (r'transmission_2.csv', index = False, header=True) # writing data in the csv file "transmission_2"

In [ ]:
#@title Analysis of the Minor IPs { display-mode: "form" }


In [ ]:
for j in minor_ips:
  dest_list_of_minor_ip = list(data[data.Source == j].Destination.drop_duplicates()) # transmitted destination list of minor IP
  if data[data.Source == j].Length.sum() <= 13000:
    naive_df = pd.DataFrame(
    [
     (j, 
      dest_list_of_minor_ip,
      f'{data[data.Source == j].Protocol.max()}', 
      f'{data[data.Source == j].Length.sum()}', 
      'Naive'),
    ],
    columns=(
        'Source', 'Destination', 'Max_Protocol', 'Total_Length', 'Result'
        )
                 )
    print(naive_df)
    # print('==========================================================')

  else:
    suspicious_df = pd.DataFrame(
    [
     (j, 
      dest_list_of_minor_ip,
      f'{data[data.Source == j].Protocol.max()}', 
      f'{data[data.Source == j].Length.sum()}', 
      'Suspicious'),
    ],
    columns=(
        'Source', 'Destination', 'Max_Protocol', 'Total_Length', 'Result'
        )
                 )
    print(suspicious_df)
    # print('==========================================================')
  


         Source  Destination Max_Protocol Total_Length Result
0  192.168.43.1  [10.0.2.15]          DNS         3047  Naive
          Source  Destination Max_Protocol Total_Length Result
0  13.227.214.62  [10.0.2.15]      TLSv1.3         4594  Naive
       Source  Destination Max_Protocol Total_Length Result
0  52.84.6.10  [10.0.2.15]      TLSv1.3         5653  Naive
       Source  Destination Max_Protocol Total_Length Result
0  52.84.6.14  [10.0.2.15]      TLSv1.3         9002  Naive
           Source  Destination Max_Protocol Total_Length Result
0  34.215.101.125  [10.0.2.15]      TLSv1.2         8481  Naive
          Source  Destination Max_Protocol Total_Length Result
0  117.18.237.29  [10.0.2.15]          TCP         2906  Naive
          Source  Destination Max_Protocol Total_Length Result
0  104.18.10.207  [10.0.2.15]      TLSv1.3         3852  Naive
         Source  Destination Max_Protocol Total_Length Result
0  104.16.19.94  [10.0.2.15]      TLSv1.3         3923  Naive
      